# Remove duplication

```mermaid
---
title: removeDuplicates.md
---
flowchart TD
    R1[(fqR1)] --> RD[removeDuplicates.md]
    R2[(fqR2)] --> RD
    RN[(...)] --> RD
    RD --> UNIQUE[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>...</th>
                <th>#</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
```

In [8]:
!removeDuplicates.md \
    remove_duplication/R2.fq.gz \
    remove_duplication/R1.fq.gz \
    > remove_duplication/unique

!removeDuplicates.md \
    remove_duplication/R1.fq.gz \
    > remove_duplication/unique_R1_only

# Demultiplex

```mermaid
---
title: demultiplex.md
---
flowchart TD
    UNIQUE[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>#</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    UNIQUE --> DM[demultiplex.md]
    MARKER[(
        <table>
            <tr>
                <th>marker1</th>
                <th>marker2</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    MARKER --> DM
    DM --> ONTARGET[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>#</th>
                <th>id</th>
                <th>...</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]

In [2]:
!spliterIndices=demultiplex/marker2,demultiplex/marker1 \
minScores=30,100 \
demultiplex.md \
    demultiplex/unique \
    > demultiplex/ontarget

# Get query

```mermaid
---
title: get query
---
flowchart TD
    ONTARGET[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>#</th>
                <th>id</th>
                <th>...</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )] --> QUERY[(
        <table>
            <tr>
                <th>query</th>
                <th>#</th>
                <th>id</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
```

adapter(20bp) + sgRNA(20bp) + scaffold(83/93bp) + target(44bp) + 3bp + RCbarcode(18bp) + RCprimer(21bp)


In [3]:
!sxCutR2AdapterFilterCumulate.md \
    get_query/ontarget \
    30 \
    > get_query/query

# Workflow

```mermaid
---
title: Workflow
---
flowchart TD
    R1[(fqR1)] --> RD[removeDuplicates.md]
    R2[(fqR2)] --> RD
    RD --> UNIQUE[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>#</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    UNIQUE --> DM[demultiplex.md]
    MARKER[(
        <table>
            <tr>
                <th>marker1</th>
                <th>marker2</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    MARKER --> DM
    DM --> ONTARGET[(
        <table>
            <tr>
                <th>R1</th>
                <th>R2</th>
                <th>#</th>
                <th>id</th>
                <th>...</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    ONTARGET --> QUERY[(
        <table>
            <tr>
                <th>query</th>
                <th>#</th>
                <th>id</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )]
    QUERY --> REARR[rearrangement]
    REF[(
        <table>
            <tr>
                <th>s1</th>
                <th>ref1</th>
                <th>e1</th>
                <th>s2</th>
                <th>ref2</th>
                <th>e2</th>
            </tr>
            <tr>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
                <td>...</td>
            </tr>
        </table>
    )] --> REARR

    REF --> CMH[correct_micro_homology.AWK]
    DIRECTION[(
        <table>
            <tr>
                <th>Up/Down</th>
            </tr>
            <tr>
                <td>...</td>
            </tr>
        </table>
    )] -->  CMH
    REARR -->  CMH
    CMH --> CORRECTED[(
        <table>
            <tr>
                <td>idx</td>
                <td>#</td>
                <td>score</td>
                <td>id</td>
                <td>...</td>
            </tr>
            <tr>
                <td>ref</td>
            </tr>
            <tr>
                <td>query</td>
            </tr>
        </table>
    )]
```

In [ ]:
!removeDuplicates.md \
    workflow/R2.fq.gz \
    workflow/R1.fq.gz \
    > workflow/unique

!spliterIndices=workflow/marker2,workflow/marker1 \
minScores=30,100 \
demultiplex.md \
    workflow/unique \
    > workflow/ontarget

!sxCutR2AdapterFilterCumulate.md \
    workflow/ontarget \
    30 \
    > workflow/query

!rearrangement \
    < workflow/query \
    3< workflow/ref \
    -s0 -6 \
    -s1 4 \
    -s2 2 \
    -u -3 \
    -v -9 \
    -ru 0 \
    -rv 0 \
    -qu 0 \
    -qv -5 | \
gawk -f correct_micro_homology.awk -- \
    workflow/ref \
    workflow/direction \
    > workflow/corrected
